In [26]:
import pandas as pd
import numpy as np

In [27]:
data=pd.read_csv('../Q1/男胎检测数据_预处理后.csv')
data

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,Y染色体浓度,X染色体浓度,13号染色体的GC含量,18号染色体的GC含量,21号染色体的GC含量,被过滤掉读段数的比例,染色体的非整倍体,怀孕次数,生产次数,胎儿是否健康
0,1,A001,31,160.0,72.0,2023-02-01,自然受孕,2023-04-29,1,11.86,...,0.025936,0.038061,0.377069,0.389803,0.399399,0.027484,NaN,1,0,是
1,2,A001,31,160.0,73.0,2023-02-01,自然受孕,2023-05-31,2,15.86,...,0.034887,0.059572,0.371542,0.384771,0.391706,0.019617,NaN,1,0,是
2,3,A001,31,160.0,73.0,2023-02-01,自然受孕,2023-06-25,3,20.14,...,0.066171,0.075995,0.377449,0.390582,0.399480,0.022312,NaN,1,0,是
3,4,A001,31,160.0,74.0,2023-02-01,自然受孕,2023-07-16,4,22.86,...,0.061192,0.052305,0.375613,0.389251,0.397212,0.023280,NaN,1,0,是
4,5,A002,32,149.0,74.0,2023-11-09,自然受孕,2024-02-19,1,13.86,...,0.059230,0.059708,0.380260,0.393618,0.404868,0.024212,NaN,2,1,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,1078,A266,30,159.0,83.4,NaN,自然受孕,2023-05-02,4,17.71,...,0.099052,0.056686,0.376861,0.389914,0.397090,0.017951,T18,1,0,是
1078,1079,A267,28,155.0,73.8,NaN,自然受孕,2023-05-17,1,11.57,...,0.098706,0.023663,0.377597,0.387901,0.404293,0.022549,T21,1,0,是
1079,1080,A267,28,155.0,74.1,NaN,自然受孕,2023-05-24,2,12.57,...,0.102088,0.080264,0.379041,0.391748,0.400433,0.021330,NaN,1,0,是
1080,1081,A267,28,155.0,74.7,NaN,自然受孕,2023-05-31,3,13.57,...,0.109855,0.074050,0.379107,0.388544,0.401030,0.022013,NaN,1,0,是


# 筛选与字段规范

In [28]:
data=data[['孕妇代码','检测孕周','孕妇BMI','Y染色体浓度']]
data

,孕妇代码,检测孕周,孕妇BMI,Y染色体浓度
0,A001,11.86,28.125000,0.025936
1,A001,15.86,28.515625,0.034887
2,A001,20.14,28.515625,0.066171
3,A001,22.86,28.906250,0.061192
4,A002,13.86,33.331832,0.059230
...,...,...,...,...
1077,A266,17.71,32.969881,0.099052
1078,A267,11.57,30.703133,0.098706
1079,A267,12.57,30.825814,0.102088
1080,A267,13.57,31.107551,0.109855


In [29]:
data=data[(data['检测孕周']>=10) & (data['检测孕周']<26)]
data.describe()

,检测孕周,孕妇BMI,Y染色体浓度
count,1073.000000,1073.000000,1073.000000
mean,16.761808,32.283321,0.077089
std,3.987793,2.972165,0.033607
min,11.000000,20.703125,0.010004
25%,13.290000,30.185195,0.051152
50%,16.000000,31.826510,0.074872
75%,20.000000,33.932236,0.098862
max,25.710000,46.875000,0.234218


## 首次达标区间构造

In [30]:
# 阈值
thr = 0.04

# 清洗与排序
data = data.copy()
data["检测孕周"] = pd.to_numeric(data["检测孕周"], errors="coerce")
data["Y染色体浓度"] = pd.to_numeric(data["Y染色体浓度"], errors="coerce")
data = data.dropna(subset=["检测孕周", "Y染色体浓度"])
data = data.sort_values(["孕妇代码", "检测孕周"], ascending=[True, True])


# 分组构造首次达标区间
def first_hit_interval(g: pd.DataFrame) -> pd.Series:
    bmi = g["孕妇BMI"].dropna().iloc[0] if g["孕妇BMI"].notna().any() else np.nan
    # convert to plain numpy float arrays to avoid ExtensionArray comparison issues
    y = g["Y染色体浓度"].to_numpy(dtype=float)
    weeks = g["检测孕周"].to_numpy(dtype=float)

    hit_idx = np.where(y >= thr)[0]
    if hit_idx.size == 0:
        # 右删失：T* > C
        C = weeks[-1]
        L, R, censor = C, np.inf, "right"
    else:
        r_pos = int(hit_idx[0])
        R = weeks[r_pos]
        if r_pos == 0:
            # 左删失：(-inf, R]
            L, censor = -np.inf, "left"
        else:
            # 区间删失：(L, R]
            L, censor = weeks[r_pos - 1], "interval"

    return pd.Series(
        {"id": g["孕妇代码"].iloc[0], "BMI": bmi, "L": L, "R": R, "censor_type": censor}
    )


intervals = (
    data.groupby("孕妇代码", group_keys=False)
    .apply(first_hit_interval)
    .reset_index(drop=True)
)

intervals

C:\Users\HP\AppData\Local\Temp\ipykernel_49024\706680993.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(first_hit_interval)


,id,BMI,L,R,censor_type
0,A001,28.125000,15.86,20.14,interval
1,A002,33.331832,-inf,13.86,left
2,A003,30.742188,-inf,13.00,left
3,A004,28.641243,-inf,11.00,left
4,A005,29.955556,-inf,12.29,left
...,...,...,...,...,...
262,A263,29.346687,-inf,12.57,left
263,A264,32.471062,-inf,11.71,left
264,A265,33.719678,-inf,11.71,left
265,A266,32.135095,-inf,13.71,left


# 条件推理树（CIT）

## 特征构造

In [31]:
# 特征构造（每人一条）

# 右删失下界代理的增量（周）
delta_week = 1.0

feat = intervals.copy()

# 删失标记：right=1，其余=0
feat["censored"] = (feat["censor_type"] == "right").astype(int)

# 达标时间代理：
# - 区间/左删失：用 R（首次达标周）
# - 右删失：用 C+Δ，这里 C=L（上一步构造里 right 情况把 L 设为最后检测周）
feat["t_proxy"] = np.where(feat["censored"] == 1, feat["L"] + delta_week, feat["R"])


# 简单 z-score 标准化（避免 std=0 的数值问题）
def zscore(s: pd.Series) -> pd.Series:
    mu = s.mean()
    sigma = s.std(ddof=0)
    sigma = 1.0 if (not np.isfinite(sigma) or sigma == 0) else sigma
    return (s - mu) / sigma


feat["x1"] = zscore(feat["BMI"])
feat["x2"] = zscore(feat["t_proxy"])

# 右删失样本较低权重
feat["sample_weight"] = np.where(feat["censored"] == 1, 0.5, 1.0)

# 输出特征表（保留原值便于核查）
features = feat[["id", "x1", "x2", "censored", "sample_weight", "BMI", "t_proxy"]]
features

,id,x1,x2,censored,sample_weight,BMI,t_proxy
0,A001,-1.246572,2.205301,0,1.0,28.125000,20.14
1,A002,0.497095,0.030966,0,1.0,33.331832,13.86
2,A003,-0.370127,-0.266793,0,1.0,30.742188,13.00
3,A004,-1.073692,-0.959256,0,1.0,28.641243,11.00
4,A005,-0.633554,-0.512617,0,1.0,29.955556,12.29
...,...,...,...,...,...,...,...
262,A263,-0.837453,-0.415672,0,1.0,29.346687,12.57
263,A264,0.208840,-0.713432,0,1.0,32.471062,11.71
264,A265,0.626977,-0.713432,0,1.0,33.719678,11.71
265,A266,0.096331,-0.020968,0,1.0,32.135095,13.71


## 分组

In [32]:
from typing import Tuple
from scipy.stats import mannwhitneyu

# 基于 BMI 的条件推理树（CIT）递归分组
df = features.copy()
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["BMI", "t_proxy"])
df = df.sort_values("BMI").reset_index(drop=True)

# 超参数
alpha = 0.05           # 显著性水平
max_depth = 5          # 最大深度
min_leaf = max(20, int(0.03 * len(df)))  # 叶子最小样本数
n_thresholds = 64      # 候选阈值个数（分位点数）

def best_split(data) -> Tuple[float, float, int, int]:
    bmis = data["BMI"].to_numpy()
    y = data["t_proxy"].to_numpy()
    if len(data) < 2 * min_leaf:
        return np.nan, 1.0, len(data), 0

    qs = np.linspace(0.1, 0.9, n_thresholds)
    cands = np.unique(np.quantile(bmis, qs))

    # 只保留左右子节点都满足 min_leaf 的阈值
    valid_cands = []
    for thr in cands:
        nl = (bmis <= thr).sum()
        nr = len(bmis) - nl
        if nl >= min_leaf and nr >= min_leaf:
            valid_cands.append(thr)

    if not valid_cands:
        return np.nan, 1.0, len(data), 0

    best_p, best_thr, best_nl, best_nr = 1.0, np.nan, 0, 0
    for thr in valid_cands:
        left = y[bmis <= thr]
        right = y[bmis > thr]
        _, p = mannwhitneyu(left, right, alternative="two-sided", method="asymptotic")
        if p < best_p:
            best_p, best_thr = p, thr
            best_nl, best_nr = len(left), len(right)

    # 返回的第二个值用作 grow 中的显著性判定，这里直接用原始 p
    return best_thr, best_p, best_nl, best_nr

leaves = []  # 存储叶子结点区间及统计
def grow(data, depth: int, lo: float, hi: float):
    thr, p_adj, nl, nr = best_split(data)
    # 停止条件：深度、无有效阈值或不显著
    if (depth >= max_depth) or (not np.isfinite(thr)) or (p_adj >= alpha):
        leaves.append({
            "lo": lo,
            "hi": hi,
            "depth": depth,
            "n": int(len(data)),
            "bmi_min": float(data["BMI"].min()),
            "bmi_max": float(data["BMI"].max()),
            "t_median": float(data["t_proxy"].median()),
        })
        return
    left = data[data["BMI"] <= thr]
    right = data[data["BMI"] > thr]
    grow(left, depth + 1, lo, thr)
    grow(right, depth + 1, thr, hi)

grow(df, depth=0, lo=-np.inf, hi=np.inf)

# 规范化叶子顺序并赋组号
leaves = sorted(leaves, key=lambda x: (x["lo"], x["hi"]))
for gid, leaf in enumerate(leaves):
    leaf["group"] = gid

def map_group_by_bmi(bmi):
    if pd.isna(bmi):
        return np.nan
    for leaf in leaves:
        if leaf["lo"] < bmi <= leaf["hi"]:
            return leaf["group"]
    # 边界兜底：若未命中则按最接近区间归类（通常不会触发）
    return len(leaves) - 1

features_cit = features.copy()
features_cit["group"] = features_cit["BMI"].apply(map_group_by_bmi)
features_cit

,id,x1,x2,censored,sample_weight,BMI,t_proxy,group
0,A001,-1.246572,2.205301,0,1.0,28.125000,20.14,0
1,A002,0.497095,0.030966,0,1.0,33.331832,13.86,0
2,A003,-0.370127,-0.266793,0,1.0,30.742188,13.00,0
3,A004,-1.073692,-0.959256,0,1.0,28.641243,11.00,0
4,A005,-0.633554,-0.512617,0,1.0,29.955556,12.29,0
...,...,...,...,...,...,...,...,...
262,A263,-0.837453,-0.415672,0,1.0,29.346687,12.57,0
263,A264,0.208840,-0.713432,0,1.0,32.471062,11.71,0
264,A265,0.626977,-0.713432,0,1.0,33.719678,11.71,0
265,A266,0.096331,-0.020968,0,1.0,32.135095,13.71,0


In [33]:
# 分组区间与统计输出
leaves_df = pd.DataFrame(leaves).sort_values(["group"]).reset_index(drop=True)

def _fmt(x):
    if not np.isfinite(x):
        return "-inf" if x < 0 else "+inf"
    return f"{x:.2f}"

leaves_df["bmi_interval"] = leaves_df.apply(
    lambda r: f"({_fmt(r['lo'])}, {_fmt(r['hi'])}]", axis=1
)

stats = (
    features_cit.groupby("group", as_index=False)
    .agg(
        n=("BMI", "size"),
        bmi_min=("BMI", "min"),
        bmi_median=("BMI", "median"),
        bmi_max=("BMI", "max"),
        t_median=("t_proxy", "median"),
        censored_rate=("censored", "mean"),
    )
    .merge(leaves_df[["group", "bmi_interval"]], on="group", how="left")
    .sort_values("group")
)

stats, features_cit

(   group    n    bmi_min  bmi_median    bmi_max  t_median  censored_rate  \
 0      0  226  20.703125   30.637769  34.484399    12.785       0.030973   
 1      1   21  34.500983   35.156250  35.944145    12.860       0.000000   
 2      2   20  36.250470   38.473206  46.875000    16.070       0.100000   
 
      bmi_interval  
 0   (-inf, 34.50]  
 1  (34.50, 36.02]  
 2   (36.02, +inf]  ,
        id        x1        x2  censored  sample_weight        BMI  t_proxy  \
 0    A001 -1.246572  2.205301         0            1.0  28.125000    20.14   
 1    A002  0.497095  0.030966         0            1.0  33.331832    13.86   
 2    A003 -0.370127 -0.266793         0            1.0  30.742188    13.00   
 3    A004 -1.073692 -0.959256         0            1.0  28.641243    11.00   
 4    A005 -0.633554 -0.512617         0            1.0  29.955556    12.29   
 ..    ...       ...       ...       ...            ...        ...      ...   
 262  A263 -0.837453 -0.415672         0            

# 最佳nipt时点

## 按组构建区间删失对象

In [34]:
# 将分组信息并入区间数据
iv = intervals.merge(features_cit[["id", "group"]], on="id", how="left")


# Turnbull NPMLE（区间删失）实现
def turnbull_npmle(
    group_df: pd.DataFrame,
    max_iter: int = 5000,
    tol: float = 1e-8,
    eps_mass: float = 1e-12,
):
    """
    输入:
      group_df 需包含列: L, R, censor_type（left/right/interval）
    返回:
      dict: {support, mass, S, F}
    """
    g = group_df.copy()

    L = g["L"].to_numpy(dtype=float)
    R = g["R"].to_numpy(dtype=float)

    # 支持点 = 所有有限端点的并集（按升序）
    finite_L = L[np.isfinite(L)]
    finite_R = R[np.isfinite(R)]
    support = (
        np.unique(np.concatenate([finite_L, finite_R]))
        if (finite_L.size + finite_R.size) > 0
        else np.array([])
    )

    n = len(g)
    K = len(support)

    # 如果没有任何有限端点（极端：全右删失或全左删失），返回常数 S(t)=1 或 0 的退化情形
    if K == 0:
        # 这里将返回一个空支持集；调用方用阶梯函数求值时会得到常数 S=1（右删失主导）
        return {
            "support": support,
            "mass": np.array([]),
            "S": np.array([]),
            "F": np.array([]),
        }

    # 每个观测对应的可行支持点索引集合 J_i
    J = []
    for Li, Ri in zip(L, R):
        if np.isneginf(Li) and np.isfinite(Ri):
            idx = np.where(support <= Ri)[0]  # 左删失：(-inf, R]
        elif np.isposinf(Ri) and np.isfinite(Li):
            idx = np.where(support > Li)[0]  # 右删失：(L, +inf)
        else:
            # 区间删失：(L, R]
            idx = np.where((support > Li) & (support <= Ri))[0]
        if idx.size == 0:
            # 若该样本与任何支持点都不相交，强制用最近的一个支持点兜底，避免空集导致算法崩溃
            # 选择与(L,R]最近的支持点
            near = np.searchsorted(support, np.nan_to_num(Ri, nan=Li))  # 尝试靠近 R
            near = min(max(0, near - 1), K - 1)
            idx = np.array([near], dtype=int)
        J.append(idx)

    # 初始化概率质量：均匀
    p = np.full(K, 1.0 / K, dtype=float)

    # EM 迭代
    for it in range(max_iter):
        counts = np.zeros(K, dtype=float)
        for idx in J:
            denom = p[idx].sum()
            if denom <= 0:
                # 退化：当前区间无质量，按均匀分配到该观测可行集
                counts[idx] += 1.0 / len(idx)
            else:
                counts[idx] += p[idx] / denom
        p_new = counts / n

        # 去除极小质量并重标化（数值稳定）
        p_new[p_new < eps_mass] = 0.0
        s = p_new.sum()
        if s <= 0:
            # 罕见退化，回退为均匀
            p_new = np.full(K, 1.0 / K, dtype=float)
            s = 1.0
        else:
            p_new = p_new / s

        # 收敛判据
        if np.linalg.norm(p_new - p, ord=1) < tol:
            p = p_new
            break
        p = p_new

    # 生存函数（右连续阶梯）：S(t_j) = 1 - sum_{k<=j} p_k
    F = np.cumsum(p)
    S = 1.0 - F

    return {
        "support": support,
        "mass": p,
        "S": S,
        "F": F,
    }


# 生存阶梯函数求值
def eval_step_S(
    t: np.ndarray, support: np.ndarray, S_support: np.ndarray
) -> np.ndarray:
    """
    在任意 t 上按阶梯函数求 S(t)。
    规则：对于 t < support[0]，S=1；对 support[j-1] < t <= support[j]，S=S_support[j]
    """
    if support.size == 0:
        return np.ones_like(t, dtype=float)
    idx = np.searchsorted(support, t, side="right") - 1
    idx = np.clip(idx, -1, len(support) - 1)
    out = np.empty_like(t, dtype=float)
    # t 在首点之前
    mask_before = idx < 0
    out[mask_before] = 1.0
    # 其他
    mask_other = ~mask_before
    out[mask_other] = S_support[idx[mask_other]]
    return out


# 构建每组的 NPMLE 结果
group_surv = {}
for g in sorted(iv["group"].dropna().unique()):
    df_g = iv[iv["group"] == g][["L", "R", "censor_type"]].reset_index(drop=True)
    res = turnbull_npmle(df_g)
    group_surv[g] = res

# 将 NPMLE 结果整理为表（支持点上）
npmle_rows = []
for g, res in group_surv.items():
    sup = res["support"]
    mass = res["mass"]
    S = res["S"]
    F = res["F"]
    if sup.size == 0:
        # 无支持点时用空行占位
        npmle_rows.append(
            pd.DataFrame(
                {
                    "group": [g],
                    "t": [np.nan],
                    "S_npmle": [1.0],
                    "F_npmle": [0.0],
                    "mass": [0.0],
                }
            )
        )
    else:
        npmle_rows.append(
            pd.DataFrame(
                {"group": g, "t": sup, "S_npmle": S, "F_npmle": F, "mass": mass}
            )
        )
npmle_df = pd.concat(npmle_rows, ignore_index=True)

# 选择统一评估网格（按题意 10~26 周，可自行调整粒度）
t_grid = np.arange(10.0, 26.0 + 1e-9, 0.5)

# 在统一网格上评估 S_g(t), F_g(t)
rows_grid = []
for g, res in group_surv.items():
    Sg = eval_step_S(t_grid, res["support"], res["S"])
    rows_grid.append(
        pd.DataFrame({"group": g, "t": t_grid, "S_npmle": Sg, "F_npmle": 1.0 - Sg})
    )
sf_grid_npmle = pd.concat(rows_grid, ignore_index=True)

# —— 离散时间生存模型（cloglog / logit 危害）用于平滑验证 ——
# 用 NPMLE 推导离散 hazard: h_j = p_j / S_{j-1}; 以 at_risk_j = n * S_{j-1} 作为试验次数（频数权重）
try:
    import statsmodels.api as sm

    has_sm = True
except Exception as e:
    has_sm = False
    print("未检测到 statsmodels，已跳过平滑验证。若需运行：pip install statsmodels")

smooth_rows = []
if has_sm:
    for g, res in group_surv.items():
        sup = res["support"]
        p = res["mass"]
        S = res["S"]

        if sup.size == 0:
            # 无支持点：S 恒为 1
            smooth_rows.append(
                pd.DataFrame(
                    {"group": g, "t": t_grid, "S_cloglog": 1.0, "S_logit": 1.0}
                )
            )
            continue

        # 计算离散 hazard 与风险集大小
        # S_prev[0]=1, S_prev[j]=S[j-1]
        S_prev = np.concatenate([[1.0], S[:-1]])
        # 避免除零
        S_prev = np.clip(S_prev, 1e-12, 1.0)
        h = p / S_prev
        h = np.clip(h, 1e-9, 1 - 1e-9)

        # 以“比例 + 频数权重”的形式拟合 GLM
        # X: [1, t, t^2] 轻度平滑（可换成更高次或样条）
        X = np.column_stack([np.ones_like(sup), sup, sup**2])

        # at-risk 人数用 n * S_prev，n = 组内样本量
        n_group = int(iv[iv["group"] == g].shape[0])
        at_risk = np.maximum(1.0, n_group * S_prev)  # 至少为 1，稳定拟合

        # cloglog
        glm_cloglog = sm.GLM(
            h,
            X,
            family=sm.families.Binomial(link=sm.families.links.cloglog()),
            freq_weights=at_risk,
        ).fit(disp=0)

        # logit
        glm_logit = sm.GLM(
            h,
            X,
            family=sm.families.Binomial(link=sm.families.links.logit()),
            freq_weights=at_risk,
        ).fit(disp=0)

        # 预测在统一网格上的 hazard，并累乘得到 S(t)
        Xg = np.column_stack([np.ones_like(t_grid), t_grid, t_grid**2])
        h_cl = np.clip(glm_cloglog.predict(Xg), 1e-9, 1 - 1e-9)
        h_lo = np.clip(glm_logit.predict(Xg), 1e-9, 1 - 1e-9)

        # S(t_k) = ∏_{j<=k} (1 - h_j)
        S_cl = np.cumprod(1.0 - h_cl)
        S_lo = np.cumprod(1.0 - h_lo)

        smooth_rows.append(
            pd.DataFrame({"group": g, "t": t_grid, "S_cloglog": S_cl, "S_logit": S_lo})
        )

smooth_df = pd.concat(smooth_rows, ignore_index=True) if smooth_rows else pd.DataFrame()

# 合并网格上的 NPMLE 与平滑结果，便于对比
if not smooth_df.empty:
    compare_df = sf_grid_npmle.merge(smooth_df, on=["group", "t"], how="left")
else:
    compare_df = sf_grid_npmle.copy()

# 输出：
# npmle_df：各组在支持点上的 S/F 和概率质量
# sf_grid_npmle：各组在统一网格(10~26, 步长0.5)上的 S/F（阶梯评估）
# compare_df：若安装 statsmodels，则包含 S_cloglog / S_logit 的平滑对比
npmle_df, sf_grid_npmle, compare_df
# ...existing code...

d:\Develop\miniconda\envs\mcm\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The cloglog link alias is deprecated. Use CLogLog instead. The cloglog link alias will be removed after the 0.15.0 release.
  warnings.warn(
d:\Develop\miniconda\envs\mcm\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The logit link alias is deprecated. Use Logit instead. The logit link alias will be removed after the 0.15.0 release.
  warnings.warn(
d:\Develop\miniconda\envs\mcm\Lib\site-packages\statsmodels\genmod\families\links.py:989: RuntimeWarning: overflow encountered in exp
  return 1 - np.exp(-np.exp(z))


(    group      t       S_npmle   F_npmle      mass
 0       0  11.00  1.751738e-01  0.824826  0.824826
 1       0  11.14  1.751738e-01  0.824826  0.000000
 2       0  11.29  1.751738e-01  0.824826  0.000000
 3       0  11.43  1.751738e-01  0.824826  0.000000
 4       0  11.57  1.751738e-01  0.824826  0.000000
 ..    ...    ...           ...       ...       ...
 91      2  20.29  1.191987e-01  0.880801  0.000000
 92      2  20.57  1.191987e-01  0.880801  0.000000
 93      2  22.00  1.191987e-01  0.880801  0.000000
 94      2  23.57  1.191987e-01  0.880801  0.000000
 95      2  24.00  1.110223e-16  1.000000  0.119199
 
 [96 rows x 5 columns],
     group     t       S_npmle   F_npmle
 0       0  10.0  1.000000e+00  0.000000
 1       0  10.5  1.000000e+00  0.000000
 2       0  11.0  1.751738e-01  0.824826
 3       0  11.5  1.751738e-01  0.824826
 4       0  12.0  1.751738e-01  0.824826
 ..    ...   ...           ...       ...
 94      2  24.0  1.110223e-16  1.000000
 95      2  24.5  1.11

## 求最早达标时点

In [35]:
# 阈值 π*（可调整）
pi_star = 0.95


# 点估计：直接基于前面计算好的 sf_grid_npmle（含每组在统一网格上的 F_npmle）
def earliest_t_from_grid(df_grid: pd.DataFrame, pi_star: float) -> pd.DataFrame:
    out = []
    for g, d in df_grid.groupby("group", sort=True):
        d = d.sort_values("t")
        mask = d["F_npmle"] >= pi_star
        if mask.any():
            t_star = float(d.loc[mask, "t"].iloc[0])
        else:
            t_star = np.nan  # 网格上尚未达到阈值
        out.append({"group": g, "t_star": t_star})
    return pd.DataFrame(out)


tstar_hat_df = earliest_t_from_grid(sf_grid_npmle, pi_star)
tstar_hat_df

,group,t_star
0,0,20.5
1,1,17.0
2,2,24.0


In [37]:
# 提速版 Bootstrap：多核并行 + 进度条
try:
    from tqdm.auto import tqdm

    _has_tqdm = True
except Exception:
    _has_tqdm = False

    def tqdm(x, **kwargs):
        return x


try:
    from joblib import Parallel, delayed

    _has_joblib = True
except Exception:
    _has_joblib = False


def bootstrap_tstar(
    iv_df: pd.DataFrame,
    B: int = 500,
    pi_star: float = 0.95,
    t_grid: np.ndarray = None,
    random_state: int = 42,
    n_jobs: int = 6,  # -1 使用全部CPU；1为单核
):
    """
    加速策略：
    - 预生成每组的索引矩阵 (B, m)，避免循环中反复调用随机采样
    - 并行按replicate维度划分任务（每个replicate内部遍历各组）
    - 进度显示：tqdm
    """
    rng = np.random.default_rng(random_state)
    if t_grid is None:
        t_grid = np.arange(10.0, 26.0 + 1e-9, 0.5)

    groups = sorted(iv_df["group"].dropna().unique())
    # 每组仅保留“每位孕妇一行”的(L,R)（censor_type由L/R含无穷表示）
    grp_data = {}
    idx_mats = {}
    for g in groups:
        iv_g = (
            iv_df[iv_df["group"] == g][["id", "L", "R"]]
            .dropna(subset=["L", "R"])
            .drop_duplicates(subset=["id"])
            .reset_index(drop=True)
        )
        L = iv_g["L"].to_numpy(dtype=float)
        R = iv_g["R"].to_numpy(dtype=float)
        m = len(L)
        grp_data[g] = {"L": L, "R": R}
        if m == 0:
            idx_mats[g] = None
        else:
            # 预生成 B 组采样下标（每次抽m个，有放回）
            idx_mats[g] = rng.integers(0, m, size=(B, m))

    def boot_once(j: int) -> np.ndarray:
        """单次replicate：返回按groups顺序的 t*_g 数组"""
        tstars = []
        for g in groups:
            data = grp_data[g]
            idx_mat = idx_mats[g]
            if idx_mat is None:
                tstars.append(np.nan)
                continue
            idx = idx_mat[j]
            Lb = data["L"][idx]
            Rb = data["R"][idx]
            # 该replicate的NPMLE
            boot_df = pd.DataFrame({"L": Lb, "R": Rb})
            res = turnbull_npmle(boot_df)
            # 网格上评估F并取最早达标时点
            Sg = eval_step_S(t_grid, res["support"], res["S"])
            Fg = 1.0 - Sg
            hit = np.where(Fg >= pi_star)[0]
            t_star = float(t_grid[hit[0]]) if hit.size > 0 else np.nan
            tstars.append(t_star)
        return np.array(tstars, dtype=float)

    # 并行或顺序执行，带进度条
    it = range(B)
    it = tqdm(it, desc="Bootstrap", unit="rep") if _has_tqdm else it

    if _has_joblib and (n_jobs != 1):
        res_list = Parallel(n_jobs=n_jobs, prefer="processes")(
            delayed(boot_once)(j) for j in it
        )
    else:
        res_list = [boot_once(j) for j in it]

    # 汇总为矩阵 (B, G)
    res_mat = np.vstack(res_list) if len(res_list) > 0 else np.empty((0, len(groups)))
    # 95%分位（忽略NaN）
    rows, boots = [], {}
    for gi, g in enumerate(groups):
        col = res_mat[:, gi] if res_mat.size > 0 else np.array([])
        boots[g] = col.tolist()
        col_valid = col[np.isfinite(col)]
        if col_valid.size == 0:
            lo = hi = np.nan
        else:
            lo, hi = np.nanpercentile(col_valid, [2.5, 97.5])
        rows.append({"group": g, "t_star_lo": lo, "t_star_hi": hi})
    ci_df = pd.DataFrame(rows)
    return ci_df, boots


# 运行 bootstrap（多核 + 进度条）
ci_df, boot_store = bootstrap_tstar(
    iv, B=500, pi_star=pi_star, t_grid=t_grid, random_state=2025, n_jobs=-1
)

# 合并点估计与区间
tstar_summary = (
    tstar_hat_df.merge(ci_df, on="group", how="left")
    .sort_values("group")
    .reset_index(drop=True)
)
tstar_summary

Bootstrap:   0%|          | 0/500 [00:00<?, ?rep/s]

,group,t_star,t_star_lo,t_star_hi
0,0,20.5,14.0000,21.0
1,1,17.0,11.7375,21.0
2,2,24.0,16.0000,24.0


# 门槛/误差稳健性

In [38]:
# ...existing code...
# ========= 门槛/误差稳健性：thr 敏感性 + 保守阈值校正 =========


def build_intervals_for_thr(df_raw: pd.DataFrame, thr_val: float) -> pd.DataFrame:
    """按给定阈值 thr_val 重新构造 (L,R] 区间；返回列: id, BMI, L, R, censor_type"""
    df = df_raw.copy()
    # 确保类型正确（幂等）
    df["检测孕周"] = pd.to_numeric(df["检测孕周"], errors="coerce")
    df["Y染色体浓度"] = pd.to_numeric(df["Y染色体浓度"], errors="coerce")
    df = df.dropna(subset=["检测孕周", "Y染色体浓度"])
    df = df.sort_values(["孕妇代码", "检测孕周"])

    def _first_hit(g: pd.DataFrame) -> pd.Series:
        bmi = g["孕妇BMI"].dropna().iloc[0] if g["孕妇BMI"].notna().any() else np.nan
        y = g["Y染色体浓度"].to_numpy(dtype=float)
        weeks = g["检测孕周"].to_numpy(dtype=float)
        hit_idx = np.where(y >= thr_val)[0]
        if hit_idx.size == 0:
            C = weeks[-1]
            L, R, censor = C, np.inf, "right"
        else:
            r_pos = int(hit_idx[0])
            R = weeks[r_pos]
            if r_pos == 0:
                L, censor = -np.inf, "left"
            else:
                L, censor = weeks[r_pos - 1], "interval"
        return pd.Series(
            {
                "id": g["孕妇代码"].iloc[0],
                "BMI": bmi,
                "L": L,
                "R": R,
                "censor_type": censor,
            }
        )

    return (
        df.groupby("孕妇代码", group_keys=False)
        .apply(_first_hit)
        .reset_index(drop=True)
    )


def recompute_tstar_for_thr(thr_val: float, col_name: str) -> pd.DataFrame:
    """在固定分组(features_cit)下，按 thr_val 重算 t*_g；返回列: group, col_name"""
    intervals_thr = build_intervals_for_thr(data, thr_val)
    iv_thr = intervals_thr.merge(
        features_cit[["id", "group"]], on="id", how="left"
    ).dropna(subset=["group"])
    # 每组 Turnbull NPMLE
    group_surv_thr = {}
    for g in sorted(iv_thr["group"].dropna().unique()):
        df_g = iv_thr[iv_thr["group"] == g][["L", "R", "censor_type"]].reset_index(
            drop=True
        )
        res = turnbull_npmle(df_g)
        group_surv_thr[g] = res
    # 网格上评估并求最早达标
    rows = []
    for g, res in group_surv_thr.items():
        Sg = eval_step_S(t_grid, res["support"], res["S"])
        Fg = 1.0 - Sg
        d = pd.DataFrame({"group": g, "t": t_grid, "F_npmle": Fg})
        rows.append(d)
    grid = (
        pd.concat(rows, ignore_index=True)
        if rows
        else pd.DataFrame(columns=["group", "t", "F_npmle"])
    )
    tstar = earliest_t_from_grid(grid, pi_star)
    tstar = tstar.rename(columns={"t_star": col_name})
    return tstar


# 1) 基线 thr=0.040 的点估计已在 tstar_hat_df 中，改名以便对照
sens_df = tstar_hat_df.rename(columns={"t_star": "t_star_thr0040"})

# 2) 其他两档阈值
for thr_val, cname in [(0.038, "t_star_thr0038"), (0.042, "t_star_thr0042")]:
    sens_df = sens_df.merge(
        recompute_tstar_for_thr(thr_val, cname), on="group", how="left"
    )

# 3) 保守阈值校正：thr_adj = 0.04 + 0.5 * σ（如有 σ 估计则填写，None 表示跳过）
sigma_est = None  # TODO: 若已估计测量误差标准差(单位同浓度)，在此填写，如 0.004
thr_adj = None if (sigma_est is None) else (0.04 + 0.5 * float(sigma_est))
if thr_adj is not None:
    sens_df = sens_df.merge(
        recompute_tstar_for_thr(thr_adj, "t_star_thr_adj"), on="group", how="left"
    )

# 4) 差异报告（单位：周；正值表示相对基线更晚达标）
sens_df["diff_0038_vs_0040"] = sens_df["t_star_thr0038"] - sens_df["t_star_thr0040"]
sens_df["diff_0042_vs_0040"] = sens_df["t_star_thr0042"] - sens_df["t_star_thr0040"]
if thr_adj is not None:
    sens_df["diff_adj_vs_0040"] = sens_df["t_star_thr_adj"] - sens_df["t_star_thr0040"]

print(
    f"thr 敏感性完成。保守阈值 thr_adj = {thr_adj:.6f}"
    if thr_adj is not None
    else "thr 敏感性完成（未提供 σ，跳过保守阈值校正）。"
)
sens_df.sort_values("group").reset_index(drop=True)
# ...existing code...

C:\Users\HP\AppData\Local\Temp\ipykernel_49024\3446158721.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_first_hit)
C:\Users\HP\AppData\Local\Temp\ipykernel_49024\3446158721.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_first_hit)
C:\Users\HP\AppData\Local\Temp\ipykernel_49024\3446158721.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprec

thr 敏感性完成（未提供 σ，跳过保守阈值校正）。


,group,t_star_thr0040,t_star_thr0038,t_star_thr0042,diff_0038_vs_0040,diff_0042_vs_0040
0,0,20.5,16.5,20.0,-4.0,-0.5
1,1,17.0,20.0,17.0,3.0,0.0
2,2,24.0,24.0,24.0,0.0,0.0


# 结果输出

In [39]:
def build_group_report(
    stats_df: pd.DataFrame,
    tstar_df: pd.DataFrame,
    sf_grid_df: pd.DataFrame,
    t_grid: np.ndarray,
    pi_star: float = 0.95,
) -> pd.DataFrame:
    # 基础合并：组信息 + t*_g 与 CI
    base = (
        stats_df[["group", "bmi_interval", "n", "censored_rate"]]
        .merge(tstar_df, on="group", how="left")
        .sort_values("group")
        .reset_index(drop=True)
    )

    # 逐组计算 F_g(t*_g) 或 t_max/F_g(t_max)
    t_max = float(np.max(t_grid))
    out = []
    for _, r in base.iterrows():
        g = r["group"]
        d = sf_grid_df[sf_grid_df["group"] == g].sort_values("t")
        t_star = r.get("t_star", np.nan)
        if np.isfinite(t_star):
            # 精确/容差匹配
            i = np.argmin(np.abs(d["t"].to_numpy() - t_star))
            F_at_tstar = float((1.0 - d["S_npmle"].iloc[i]))
            reached = F_at_tstar >= pi_star - 1e-12
            out.append(
                {
                    "group": g,
                    "F_at_t_star": F_at_tstar,
                    "t_max": np.nan,
                    "F_at_t_max": np.nan,
                    "reached": reached,
                    "advice": "" if reached else "建议延后一周或复检",
                }
            )
        else:
            # 未达到阈值，取最大网格点
            F_at_tmax = float((1.0 - d["S_npmle"].iloc[-1])) if len(d) else np.nan
            out.append(
                {
                    "group": g,
                    "F_at_t_star": np.nan,
                    "t_max": t_max,
                    "F_at_t_max": F_at_tmax,
                    "reached": False,
                    "advice": "t_max仍未达95%，建议延后一周或复检",
                }
            )

    extra = pd.DataFrame(out)
    rep = base.merge(extra, on="group", how="left")

    # 格式化列
    rep["N"] = rep["n"].astype(int)
    rep["删失率(%)"] = (rep["censored_rate"] * 100).round(1)
    rep["t*_g(周)"] = rep["t_star"].round(2)
    rep["95%CI_low"] = rep["t_star_lo"].round(2)
    rep["95%CI_high"] = rep["t_star_hi"].round(2)
    rep["F_g(t*_g)"] = rep["F_at_t_star"].round(3)
    rep["t_max(周)"] = rep["t_max"].round(2)
    rep["F_g(t_max)"] = rep["F_at_t_max"].round(3)

    # 选择并重命名展示列
    cols = [
        "group",
        "bmi_interval",
        "N",
        "删失率(%)",
        "t*_g(周)",
        "95%CI_low",
        "95%CI_high",
        "F_g(t*_g)",
        "t_max(周)",
        "F_g(t_max)",
        "advice",
    ]
    rep = rep[cols].rename(columns={"group": "组号", "bmi_interval": "BMI区间"})
    return rep


group_report = build_group_report(
    stats, tstar_summary, sf_grid_npmle, t_grid, pi_star=pi_star
)
group_report

,组号,BMI区间,N,删失率(%),t*_g(周),95%CI_low,95%CI_high,F_g(t*_g),t_max(周),F_g(t_max),advice
0,0,"(-inf, 34.50]",226,3.1,20.5,14.00,21.0,0.978,NaN,NaN,
1,1,"(34.50, 36.02]",21,0.0,17.0,11.74,21.0,1.000,NaN,NaN,
2,2,"(36.02, +inf]",20,10.0,24.0,16.00,24.0,1.000,NaN,NaN,


In [ ]:
# ...existing code...
# ========= 可视化与方法对照 =========
import matplotlib.pyplot as plt
from math import ceil
import matplotlib as mpl

# Force English default fonts to avoid missing glyphs for CJK
mpl.rcParams.update({
    "font.family": "DejaVu Sans",
    "font.sans-serif": ["DejaVu Sans", "Arial", "Liberation Sans", "sans-serif"],
    "axes.unicode_minus": False,
})


# 1) Bootstrap 生成 F_g(t) 置信带（百分位法）
def bootstrap_F_bands(
    iv_df: pd.DataFrame,
    t_grid: np.ndarray,
    B: int = 300,
    random_state: int = 2025,
    q_low: float = 2.5,
    q_high: float = 97.5,
):
    rng = np.random.default_rng(random_state)
    groups = sorted(iv_df["group"].dropna().unique())
    T = len(t_grid)

    # 预先收集每组样本
    grp_arrays = {}
    for g in groups:
        d = (
            iv_df[iv_df["group"] == g][["id", "L", "R"]]
            .dropna(subset=["L", "R"])
            .drop_duplicates(subset=["id"])
            .reset_index(drop=True)
        )
        grp_arrays[g] = (d["L"].to_numpy(float), d["R"].to_numpy(float))

    # 存放每组 F 曲线的矩阵 (B, T)
    F_store = {g: np.full((B, T), np.nan, dtype=float) for g in groups}

    for j in range(B):
        for g in groups:
            L, R = grp_arrays[g]
            m = len(L)
            if m == 0:
                continue
            idx = rng.integers(0, m, size=m)
            Lb, Rb = L[idx], R[idx]
            res = turnbull_npmle(pd.DataFrame({"L": Lb, "R": Rb}))
            Sg = eval_step_S(t_grid, res["support"], res["S"])
            F_store[g][j, :] = 1.0 - Sg

    # 汇总为 (group, t, F_lo, F_med, F_hi)
    rows = []
    for g in groups:
        M = F_store[g]
        # 对每个 t 的 B 次重采样取分位
        F_lo = np.nanpercentile(M, q_low, axis=0)
        F_md = np.nanpercentile(M, 50.0, axis=0)
        F_hi = np.nanpercentile(M, q_high, axis=0)
        rows.append(
            pd.DataFrame(
                {"group": g, "t": t_grid, "F_lo": F_lo, "F_med": F_md, "F_hi": F_hi}
            )
        )
    bands = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
    return bands


# 2) 达标曲线图：F_g(t) 台阶 + 置信带 + t*_g 标注
def plot_F_with_bands(sf_grid_df, bands_df, tstar_df, stats_df, t_grid):
    groups = sorted(sf_grid_df["group"].dropna().unique())
    G = len(groups)
    ncols = 2 if G <= 6 else 3
    nrows = ceil(G / ncols)
    fig, axes = plt.subplots(
        nrows, ncols, figsize=(6 * ncols, 4.2 * nrows), sharex=True, sharey=True
    )
    if G == 1:
        axes = np.array([axes])
    axes = axes.ravel()

    # 组标签（BMI 区间 + N）
    label_map = (
        stats_df.set_index("group")[["bmi_interval", "n"]]
        .apply(lambda r: f"{r.name}: {r['bmi_interval']} | N={int(r['n'])}", axis=1)
        .to_dict()
    )

    for ax, g in zip(axes, groups):
        d_line = sf_grid_df[sf_grid_df["group"] == g].sort_values("t")
        ax.step(
            d_line["t"],
            d_line["F_npmle"],
            where="post",
            linewidth=2.0,
            label="F_g(t) NPMLE",
        )

        if (bands_df is not None) and (not bands_df.empty):
            d_band = bands_df[bands_df["group"] == g].sort_values("t")
            ax.fill_between(
                d_band["t"],
                d_band["F_lo"],
                d_band["F_hi"],
                color="#5dade2",
                alpha=0.25,
                label="95% band",
            )
            ax.plot(
                d_band["t"],
                d_band["F_med"],
                color="#1f77b4",
                alpha=0.7,
                linestyle="--",
                label="bootstrap median",
            )

        # t*_g 标注
        ts = tstar_df[tstar_df["group"] == g]["t_star"]
        if len(ts) and np.isfinite(ts.iloc[0]):
            t_star = float(ts.iloc[0])
            ax.axvline(
                t_star,
                color="#e74c3c",
                linestyle="--",
                linewidth=1.6,
                label=f"t*_g={t_star:.2f}",
            )
            ax.text(
                t_star,
                0.02,
                f"{t_star:.2f}",
                color="#e74c3c",
                rotation=90,
                va="bottom",
                ha="right",
            )

        ax.set_title(label_map.get(g, f"group {g}"))
        ax.set_xlim(float(t_grid.min()), float(t_grid.max()))
        ax.set_ylim(0.0, 1.0)
        ax.grid(alpha=0.25)

    # 清理空子图
    for k in range(len(groups), len(axes)):
        axes[k].axis("off")

    fig.supylabel("F_g(t)")
    fig.supxlabel("Gestational week (wks)")
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="upper center", ncols=4, frameon=False)
    fig.tight_layout(rect=[0, 0, 1, 0.92])
    return fig


# 3) 方法对照：分位数等分 vs 1D-KMeans 聚类，比较 t*_g，并计算“提前周数”
def kmeans_1d(
    values: np.ndarray, k: int, max_iter: int = 100, random_state: int = 2025
):
    rng = np.random.default_rng(random_state)
    v = np.sort(values[~np.isnan(values)])
    if len(v) == 0:
        return None, None
    # 用分位点初始化中心
    qs = np.linspace(0, 1, k + 2)[1:-1]
    centers = np.quantile(v, qs)
    for _ in range(max_iter):
        # 分配
        dist = np.abs(values.reshape(-1, 1) - centers.reshape(1, -1))
        labels = np.nanargmin(dist, axis=1)
        # 更新
        new_centers = centers.copy()
        for i in range(k):
            pts = values[labels == i]
            pts = pts[~np.isnan(pts)]
            if len(pts) > 0:
                new_centers[i] = np.mean(pts)
        if np.allclose(new_centers, centers):
            break
        centers = new_centers
    # 依据中心排序，重映射标签为 0..k-1
    order = np.argsort(centers)
    remap = {old: new for new, old in enumerate(order)}
    labels_sorted = np.array([remap[l] for l in labels])
    centers_sorted = centers[order]
    return labels_sorted, centers_sorted


def compute_tstar_given_assignment(
    iv_df: pd.DataFrame, assign_df: pd.DataFrame, t_grid: np.ndarray, pi_star: float
):
    ivx = iv_df.merge(assign_df, on="id", how="inner").dropna(subset=["group"])
    group_surv = {}
    for g in sorted(ivx["group"].dropna().unique()):
        d = ivx[ivx["group"] == g][["L", "R", "censor_type"]].reset_index(drop=True)
        res = turnbull_npmle(d)
        group_surv[g] = res
    # 网格求最早达标
    rows = []
    for g, res in group_surv.items():
        Sg = eval_step_S(t_grid, res["support"], res["S"])
        Fg = 1.0 - Sg
        rows.append(pd.DataFrame({"group": g, "t": t_grid, "F_npmle": Fg}))
    grid = (
        pd.concat(rows, ignore_index=True)
        if rows
        else pd.DataFrame(columns=["group", "t", "F_npmle"])
    )
    return earliest_t_from_grid(grid, pi_star)


def build_method_compare(
    features_df: pd.DataFrame,
    intervals_df: pd.DataFrame,
    t_grid: np.ndarray,
    pi_star: float,
    n_groups: int,
):
    base = (
        features_df[["id", "BMI"]]
        .dropna()
        .drop_duplicates(subset=["id"])
        .reset_index(drop=True)
    )
    # 分位数等分
    labels_q = pd.qcut(
        base["BMI"],
        q=min(n_groups, max(1, base["BMI"].nunique())),
        labels=False,
        duplicates="drop",
    )
    base_q = base.assign(group=labels_q.astype(float)).dropna(subset=["group"])
    base_q["group"] = base_q["group"].astype(int)
    t_q = compute_tstar_given_assignment(
        intervals_df, base_q[["id", "group"]], t_grid, pi_star
    ).rename(columns={"t_star": "t_star_quantile"})

    # 1D-KMeans
    km_labels, centers = kmeans_1d(base["BMI"].to_numpy(float), n_groups)
    base_km = base.copy()
    base_km["group"] = km_labels
    t_km = compute_tstar_given_assignment(
        intervals_df, base_km[["id", "group"]], t_grid, pi_star
    ).rename(columns={"t_star": "t_star_kmeans"})

    cmp_df = (
        t_q.merge(t_km, on="group", how="outer")
        .sort_values("group")
        .reset_index(drop=True)
    )
    cmp_df["adv_weeks(kmeans_vs_quantile)"] = (
        cmp_df["t_star_quantile"] - cmp_df["t_star_kmeans"]
    )
    return cmp_df


def plot_method_compare_bars(cmp_df: pd.DataFrame):
    groups = cmp_df["group"].to_numpy()
    x = np.arange(len(groups))
    width = 0.35
    fig, ax = plt.subplots(figsize=(1.2 + 1.8 * len(groups), 4.5))
    ax.bar(x - width / 2, cmp_df["t_star_quantile"], width, label="Quantile split")
    ax.bar(x + width / 2, cmp_df["t_star_kmeans"], width, label="KMeans clusters")
    for i, adv in enumerate(cmp_df["adv_weeks(kmeans_vs_quantile)"]):
        if np.isfinite(adv):
            ax.text(
                x[i],
                np.nanmax(
                    [cmp_df["t_star_quantile"].iloc[i], cmp_df["t_star_kmeans"].iloc[i]]
                )
                + 0.1,
                f"Δ weeks {adv:+.2f}",
                ha="center",
                va="bottom",
                fontsize=9,
            )
    ax.set_xticks(x)
    ax.set_xticklabels([str(int(g)) for g in groups])
    ax.set_ylabel("t*_g (weeks)")
    ax.set_xlabel("Group")
    ax.set_title("Methods comparison: quantile split vs KMeans (t*_g)")
    ax.grid(axis="y", alpha=0.3)
    ax.legend()
    fig.tight_layout()
    return fig


# 4) 敏感性图：不同阈值下 t*_g 条形对比（需先运行“门槛/误差稳健性”单元得到 sens_df）
def plot_threshold_sensitivity_bars(sens_df: pd.DataFrame, stats_df: pd.DataFrame):
    df = sens_df.copy()
    cols = [c for c in df.columns if c.startswith("t_star_thr")]
    if not cols:
        print("t_star_thrXXXX columns not found. Please run the sensitivity cell first.")
        return None
    thr_map = {
        c: float(c.replace("t_star_thr", "")) / 1000.0 for c in cols
    }  # e.g., 0040 -> 0.040
    long_rows = []
    for _, r in df.iterrows():
        g = r["group"]
        for c in cols:
            long_rows.append({"group": g, "thr": thr_map[c], "t_star": r[c]})
    long_df = (
        pd.DataFrame(long_rows)
        .merge(stats_df[["group", "bmi_interval"]], on="group", how="left")
        .sort_values(["group", "thr"])
    )

    groups = sorted(long_df["group"].unique())
    n_thr = len(cols)
    width = 0.8 / n_thr
    fig, ax = plt.subplots(figsize=(1.2 + 2.0 * len(groups), 4.5))
    for i, thr in enumerate(sorted(long_df["thr"].unique())):
        d = long_df[long_df["thr"] == thr]
        x = np.arange(len(groups)) + (i - (n_thr - 1) / 2) * width
        ax.bar(x, d["t_star"], width, label=f"thr={thr:.3f}")
    ax.set_xticks(np.arange(len(groups)))
    ax.set_xticklabels(
        [
            f"{g}\n{stats_df.loc[stats_df['group']==g, 'bmi_interval'].values[0]}"
            for g in groups
        ],
        rotation=0,
    )
    ax.set_ylabel("t*_g (weeks)")
    ax.set_xlabel("Group | BMI interval")
    ax.set_title("Sensitivity: t*_g across thresholds")
    ax.grid(axis="y", alpha=0.3)
    ax.legend()
    fig.tight_layout()
    return fig


# ===== 运行：生成三类图形与方法对照表 =====

# A) F_g(t) 置信带（bootstrap）
bands_df = bootstrap_F_bands(iv, t_grid=t_grid, B=300, random_state=2025)
fig_fg = plot_F_with_bands(sf_grid_npmle, bands_df, tstar_summary, stats, t_grid)
plt.show()

# B) 方法对照表 + 柱状图（分位数等分 vs 聚类分组）
n_groups = stats.shape[0]
method_compare_df = build_method_compare(
    features, intervals, t_grid, pi_star, n_groups=n_groups
)
display(method_compare_df)
fig_methods = plot_method_compare_bars(method_compare_df)
plt.show()

# C) 阈值敏感性柱状图（需要 sens_df）
try:
    _ = sens_df  # 检查变量是否存在
    fig_sens = plot_threshold_sensitivity_bars(sens_df, stats)
    if fig_sens is not None:
        plt.show()
except NameError:
    print("sens_df not found. Please run the 'Robustness' (threshold sensitivity) cell first.")
# ...existing code...